In [1]:
import anndata as ad
import scanpy as sc, pandas as pd, numpy as np
import glob2, os, re
from collections import Counter
from openpyxl import load_workbook
from collections import defaultdict
# using datetime module
import datetime
import functools


In [2]:
adata = ad.read('STAR_202_samples_hvg_hpy_umap.h5ad')

In [3]:
covariates_file = pd.read_csv('All_logs.tsv', sep = "\t", header=[0, 1, 2])
covariates_file.head(3)

LAB                                                  STAR  \
  BATCH                  SAMPLE    BATCH                  REG   
  ROUND                  SAMPLE PREPARER REP        JOB_START   
0     2  NPSAD-20201117-A1-cDNA        A   1  Jul/08/10:45:40   
1     2  NPSAD-20201110-A1-cDNA        A   1  Jul/08/02:02:51   
2     2  NPSAD-20201117-C2-cDNA        C   2  Jul/11/13:38:02   

                                                                        ...  \
                                                                        ...   
         MAP_START       JOB_FINISH MAP_SPEED     N_READS AVG_READ_LEN  ...   
0  Jul/08/13:09:16  Jul/08/17:37:24    391.20  1748251072           90  ...   
1  Jul/08/03:27:24  Jul/08/06:29:52    404.36  1229717948           90  ...   
2  Jul/11/15:11:49  Jul/11/18:43:51    398.67  1408846088           90  ...   

          cellranger                                         \
               DEMUX                                          
  N_DOUBLET_CELLS_CS N_NEGATIVE_CELLS_CS N_DOUBLET_CELLS_MS   
0               3276                  61               3369   
1               5789                  46               5785   
2               5532                  52               5397   

                                                                             \
                                                                              
  N_NEGATIVE_CELLS_MS N_DOUBLET_CELLS_HD N_NEGATIVE_CELLS_HD N_CELLS_COMMON   
0                 618               3019                1442          11926   
1                1082               5688                1199          14830   
2                 473               5375                 800          15220   

                                                      \
                                                       
                              N_CELLS_AFTER_DEMUX_CS   
0  M83930:3858,M639636:3369,M8975:2656,H1560:1836...   
1  M84936:3893,M96977:3260,R86712535:2669,M74724:...   
2  M86494:3619,M38454:2965,M76678:2650,M64974:263...   

                                                      \
                                                       
                              N_CELLS_AFTER_DEMUX_MS   
0  M83930:3732,M639636:3240,M8975:2288,H1560:1858...   
1  M84936:3681,M96977:3217,M99118:2443,M74724:242...   
2  M86494:3867,M38454:2671,M76678:2586,M64974:257...   

                                                      
                                                      
                              N_CELLS_AFTER_DEMUX_HD  
0  M83930:3703,M639636:2736,M8975:2119,H1560:1959...  
1  M84936:3732,M96977:3009,M99118:2657,M99345:241...  
2  M86494:3925,M64974:2610,M76678:2549,M39282:254...  

[3 rows x 189 columns]

In [4]:
adata

AnnData object with n_obs × n_vars = 3518365 × 6160
    obs: 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mito', 'log1p_total_counts_mito', 'pct_counts_mito', 'batch', 'round_num', 'prep', 'rep', 'SubID_hd', 'HTO_n_hd', 'SubID_cs', 'SubID_ms', 'HTO_n_cs', 'HTO_n_ms', 'batch_1'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'batch_1_colors', 'hvg', 'neighbors', 'pca', 'prep_colors', 'umap'
    obsm: 'X_pca', 'X_pca_harmony', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [8]:
vireo_inp = pd.read_csv('samples_for_vireo.txt', names=['samples'])

In [13]:
vireo_inp.head(3)

,samples
0,NPSAD-20201015-C1-cDNA
1,NPSAD-20201019-C1-cDNA
2,NPSAD-20201020-A1-cDNA


In [15]:
adata.obs['batch_1'] = adata.obs['batch'].map(lambda x: x[:-6])

In [39]:
adata.obs_names[adata.obs['batch'] == 'NPSAD-20201015-C1-cDNA'].tolist()

['AAACCCAAGCGAAACC',
 'AAACCCAAGCGAGTCA',
 'AAACCCACAGGCGAAT',
 'AAACCCAGTATCGTGT-1',
 'AAACCCAGTTCGGTTA',
 'AAACCCATCATCCTAT-1',
 'AAACCCATCCACCTGT',
 'AAACCCATCGAGAGCA',
 'AAACCCATCGGACCAC-1',
 'AAACCCATCGGAGATG-1',
 'AAACCCATCTAAACGC-1',
 'AAACCCATCTACACAG',
 'AAACGAAAGAAGCCTG-1',
 'AAACGAAAGGAAAGGT',
 'AAACGAAAGGGACACT-2',
 'AAACGAAAGGGCTTCC-1',
 'AAACGAAAGTTGCGAG',
 'AAACGAACACTGGCCA',
 'AAACGAACATCACAGT',
 'AAACGAAGTATACAGA',
 'AAACGAAGTCACGCTG-1',
 'AAACGAAGTCACTCAA-2',
 'AAACGAAGTGTAGTGG-1',
 'AAACGAAGTTGATGTC',
 'AAACGAATCCATACTT',
 'AAACGAATCTGAGCAT',
 'AAACGCTAGAGGGTGG-1',
 'AAACGCTAGCAAATGT',
 'AAACGCTAGCCATTGT',
 'AAACGCTAGGTCTACT-1',
 'AAACGCTAGTCTGCAT-1',
 'AAACGCTAGTGGCCTC',
 'AAACGCTAGTTCAACC',
 'AAACGCTCAAAGAGTT',
 'AAACGCTCACATATGC-1',
 'AAACGCTCACGGCACT-1',
 'AAACGCTCAGAGGCAT',
 'AAACGCTCAGGAGGAG-1',
 'AAACGCTCAGGAGGTT-1',
 'AAACGCTGTGATACTC',
 'AAACGCTTCGACACTA-1',
 'AAACGCTTCGCCAGAC-2',
 'AAAGAACAGGACAACC',
 'AAAGAACAGGAGACCT',
 'AAAGAACAGTACAGAT-1',
 'AAAGAACAGTC

In [37]:
vireo_inp.dropna(inplace=True)

In [42]:
for v in vireo_inp['samples']:
    try:
        r=adata[adata.obs['batch'] == v].obs['round_num'].unique()[0]
    except:
        continue
    with open('filt_bc_vireo_{}_{}.txt'.format(r, v), 'w') as f:
        for item in adata.obs_names[adata.obs['batch'] == v].tolist():
            item=item[:16]
            f.write("%s\n" % item)
    